## Step 0: Imports

In [1]:
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
import os

In [ ]:
# Step 2: Import libraries
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
import os

# Step 3: Initialize Pinecone
api_key = "your_pinecone_api"  # 🔐 Paste your Pinecone API key
pc = Pinecone(api_key=api_key)

# Step 4: Create index (only do this once, skip if already created)
index_name = "nlp-demo2"
dimension = 384  # because we're using all-MiniLM-L6-v2

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",        # or "gcp"
            region="us-east-1"  # match your Pinecone project region
        )
    )
model = SentenceTransformer("all-MiniLM-L6-v2")

## Step 1: Replace Sentences with FAQs

In [3]:
faqs = [
    "You can reset your password from the settings page.",
    "Refunds are processed within 5–7 business days.",
    "You can upgrade your plan from the billing section.",
    "To contact support, go to the Help Center.",
    "We offer a free trial for all new users.",
]

In [5]:
faq_embeddings = model.encode(faqs)
index = pc.Index(index_name)
to_upsert = [(f"id-{i}", emb.tolist()) for i, emb in enumerate(faq_embeddings)]

index.upsert(vectors=to_upsert)

# Also build ID → answer mapping
id_to_faq = {f"id-{i}": faq for i, faq in enumerate(faqs)}

## Step 2: Semantic Search on User’s Question

In [6]:
user_question = input("💬 Ask a question: ")
query_vec = model.encode([user_question]).tolist()[0]

result = index.query(vector=query_vec, top_k=2)

print(f"\n🔍 Question: {user_question}\n")
for match in result["matches"]:
    faq_id = match["id"]
    score = match["score"]
    answer = id_to_faq.get(faq_id, "Not found")

    print(f"✅ Answer: {answer}")
    print(f"🆔 ID: {faq_id} | 🔢 Score: {score:.4f}\n")

💬 Ask a question:  how to get refund



🔍 Question: how to get refund

✅ Answer: Refunds are processed within 5–7 business days.
🆔 ID: id-1 | 🔢 Score: 0.5466

✅ Answer: To contact support, go to the Help Center.
🆔 ID: id-3 | 🔢 Score: 0.4118

